In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [17]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv('Resources/lending_data.csv')

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [18]:
# Separate the data into labels and features

# Separate the y variable, the labels
total_y = lending_data_df["loan_status"]

# Separate the X variable, the features
total_x = lending_data_df.drop(columns="loan_status")

In [19]:
# Review the y variable Series
total_y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [20]:
# Review the X variable DataFrame
total_x.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [21]:
# Check the balance of our target values
total_balance = total_y.value_counts()
print(total_balance)

0    75036
1     2500
Name: loan_status, dtype: int64


### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [22]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test, y_train, y_test = train_test_split(total_x, total_y, random_state=1)
print("Number of training features", len(x_train))
print("Number of testing features", len(x_test))
print("Number of training labels", len(y_train))
print("Number of testing labels", len(y_test))


Number of training features 58152
Number of testing features 19384
Number of training labels 58152
Number of testing labels 19384


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [23]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model.fit(x_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [24]:
# Make a prediction using the testing data
y_pred = lr_model.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [26]:
# Print the balanced_accuracy score of the model
balanced_acc_score = balanced_accuracy_score(y_pred, y_test)
print("The balanced accuracy score is", balanced_acc_score * 100, "%")

The balanced accuracy score is 92.18124642767772 %


In [27]:
# Generate a confusion matrix for the model
conf_mat = confusion_matrix(y_pred=y_pred, y_true=y_test)
print("The following is the confusion matrix: ")
print(conf_mat)

The following is the confusion matrix: 
[[18663   102]
 [   56   563]]


In [28]:
# Print the classification report for the model
class_report = classification_report(y_test, y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Overall the model does a better job predicting healthy loan labels ("0") compared to predicting high-risk loan labels ("1").  This can be shown in the report above, where the precision and f1-score for the healthy loan predictions are 1.00, whereas the precision and f1-score for the high-risk loan predictions are 0.85 and 0.88, respectively.  These results are reasonable because the model had much more support from healthy loan examples than it did from the high-risk loan examples.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [31]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
sampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
x_resample, y_resample = sampler.fit_resample(x_train, y_train)

In [32]:
# Count the distinct values of the resampled labels data
resample_counts = y_resample.value_counts()
print(resample_counts)

0    56271
1    56271
Name: loan_status, dtype: int64


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [33]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resample_lr_model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
resample_lr_model.fit(x_resample, y_resample)

# Make a prediction using the testing data
y_pred_resample = resample_lr_model.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [34]:
# Print the balanced_accuracy score of the model 
balanced_acc_score_resample = balanced_accuracy_score(y_pred=y_pred_resample, y_true=y_test)
print("The balanced accuracy score is", balanced_acc_score_resample * 100, "%")

The balanced accuracy score is 99.36781215845846 %


In [35]:
# Generate a confusion matrix for the model
conf_mat_resample = confusion_matrix(y_pred=y_pred_resample, y_true=y_test)
print("The following is the confusion matrix: ")
print(conf_mat_resample)

The following is the confusion matrix: 
[[18649   116]
 [    4   615]]


In [36]:
# Print the classification report for the model
class_report_resample = classification_report(y_test, y_pred_resample)
print(class_report_resample)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The healthy loan predictions continued to be predicted very well by the model, as the precision and f1-scores were both 1.00 again.  The high-risk loan predictions were better, as the f1-score increased to a value of 0.91, however, as seen before, the high-risk loan predictions were not as good as the healthy loan predictions.